### Setup

In [1]:
// import dependencies
var {smock} = require('@defi-wonderland/smock')

var moment = require('moment')
var { constants } = require('../utils')
var { advanceTimeAndBlock } = require('../utils/evm')
var { evm, common, contracts } = require('../utils');
var { toUnit } = require('../utils/bn')
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm')
var { next, clear, bnToNumber } = require('../utils/jupyter')
var { Notebook } = require('../utils/notebook')
var { ethers } = require('hardhat')

var $ = new Notebook();

clear();

// console.log(smocklib.smock)

In [2]:
// setup tools and contracts

next(async () => {
    await $.setup(constants.FORK_BLOCK_NUMBER);
    
    provider = await $.impersonate(constants.RICH_DAI_ADDRESS);
    
    // contract artifacts
    ls = await $.fetch('LongShort', constants.LONG_SHORT_ADDRESS)
    synthF3Long = await $.fetch('SyntheticToken', constants.SYNTHETIC_TOKEN_ADDRESS_F3_LONG)
    synthF3Short = await $.fetch('SyntheticToken',constants.SYNTHETIC_TOKEN_ADDRESS_F3_SHORT)
    ym = await $.fetch('YieldManagerAave', constants.YIELD_MANAGER_ADDRESS)
    oracleManager = await $.fetch('OracleManagerFlippening_V0', constants.ORACLE_MANAGER_ADDRESS_F3)
    alphaFLT = await $.fetch('AlphaTestFLT',constants.ALPHA_FLT_ADDRESS)
    treasury = await $.fetch('TreasuryAlpha',constants.TREASURY_ALPHA_ADDRESS)
    
    dai = await $.fetch('ERC20', constants.DAI_ADDRESS )
    
    // fake contracts
    oracleManagerFake = await smock.fake('OracleManagerFlippening_V0');
    // update the address of the oracle manager to whatever the address of the fake is.
    // might have to make myself admin in order to do this.
    longShortFake = await smock.fake('LongShort');
});

In [51]:
//updating the address of the oracle manager to the fake oracleManager contract
next(async()=>{
    ls.connect(await $.impersonate('0xcfcCB5e6b3882b7795eb1B1A9f18831A842d4eE0')).updateMarketOracle(1,oracleManagerFake.address)    
})

In [98]:
//Checking total supply of long F3 
synthF3Long.totalSupply()
/0x1b78e92b78118e7a5d6b/10**18

BigNumber { _hex: '0x1b78e92b78118e7a5d6b', _isBigNumber: true }

In [77]:
//Approving transaction for the user
dai.connect(provider).approve(ls.address,200000000000)

{
  hash: '0x147cd0cecf638c34a5de84246c428d5ba2744b208ff1a3fb2260dfabde2f7292',
  type: 0,
  accessList: null,
  blockHash: '0xc117a8af0ded19985938b9f5fde2079387bcc8caef73db1fc1b5bddc2ce2c531',
  blockNumber: 20001045,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x06959153B974D0D5fDfd87D561db6d8d4FA0bb0B',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01badb58', _isBigNumber: true },
  to: '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 3377,
  data: '0x095ea7b3000000000000000000000000168a5d1217aecd258b03018d5bf1a1677a07b7330000000000000000000000000000000000000000000000000000002e90edd000',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 27,
  creates: null,
  chainId: 0,
  wait: [Function (anonymous)]
}

In [78]:
//Mint long next price for user in F3 market
next(async()=>{
    ls.connect(provider).mintLongNextPrice(await synthF3Long.marketIndex(),200000000000)           
           })


In [80]:
ls.userNextPrice_paymentToken_depositAmount(1,true,constants.RICH_DAI_ADDRESS)

 0x8bb2c97000 / 10**18

// ls.userNextPrice_currentUpdateIndex(1,constants.RICH_DAI_ADDRESS)

6e-7

In [81]:
next(async()=>{
    tx = await ls.updateSystemState(1)
    waited = await tx.wait()
})

In [82]:
oracleManagerFake.updatePrice.getCall(1)

{
  args: [],
  nonce: 1,
  target: '0xd7899d769d69b4f79f3653e0326a21065b450d0c'
}

In [83]:
//Forcing an oracle price update for next price execution to go through
oracleManagerFake.updatePrice.returns('42000000000000000000')

In [84]:
oracleManagerFake.connect(provider).updatePrice()

{
  hash: '0x4e3e3f619571860ecfe1a64f4862dc5bdd72d240ac768e2ab9c27a583e9d57df',
  type: 0,
  accessList: null,
  blockHash: '0x6d6f4eaf3084ccd3db9b39ec33e84177025cb6aa64fe4ec89b23a66d0019f4fd',
  blockNumber: 20001048,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x06959153B974D0D5fDfd87D561db6d8d4FA0bb0B',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0xD7899D769d69B4f79f3653E0326a21065b450d0c',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 3379,
  data: '0x673a7e28',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 27,
  creates: null,
  chainId: 0,
  wait: [Function (anonymous)]
}

In [85]:
//Running next price execution for the user
ls.executeOutstandingNextPriceSettlementsUser(constants.RICH_DAI_ADDRESS,1)

{
  hash: '0xd4f807be698afe97e8e48ddf5072d66abf3dd72284ebb419281d7826e5b38277',
  type: 0,
  accessList: null,
  blockHash: '0x60fa65ff57b925f30bd53adf85e19d635ba74bce9f0c61d92243604914d4591b',
  blockNumber: 20001049,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bada98', _isBigNumber: true },
  to: '0x168a5d1217AEcd258b03018d5bF1A1677A07b733',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 789,
  data: '0x897a078600000000000000000000000006959153b974d0d5fdfd87d561db6d8d4fa0bb0b0000000000000000000000000000000000000000000000000000000000000001',
  r: '0x73fe29a4e92956130d5079264ce5ca1b3f38142f870cabc5a9483cc1391c0098',
  s: '0x48f059aeb5ee4a0e27034cc82dfb9c00099503666b83e90ecc743a738a683baf',
  v: 62710,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}

In [86]:
// balance of synthetic tokens for the user
synthF3Long.balanceOf(constants.RICH_DAI_ADDRESS)

// 0x63b81c6692 /10**18

BigNumber { _hex: '0x63b81c6692', _isBigNumber: true }

In [43]:
oracleManagerFake.connect(provider).updatePrice()

{
  hash: '0x8239a9c5927f12952f6b71832ffef8044027a917d1b30499df89e5306d283f3a',
  type: 0,
  accessList: null,
  blockHash: '0x1bab73391541b7a391c74cfe88ea5497f16e7dbf6c82fe05eaaccb5284826e93',
  blockNumber: 20001031,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x06959153B974D0D5fDfd87D561db6d8d4FA0bb0B',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x9B354Ae2bB585CAceF38f6CF7a492e31860092C5',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 3372,
  data: '0x673a7e28',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 27,
  creates: null,
  chainId: 0,
  wait: [Function (anonymous)]
}

In [15]:
oracleManager.getLatestPrice()

0x023d393ebf869096a4 / 10**18

// oracleManagerFake.getLatestPrice()

41.30511444972252

In [11]:
oracleManager.updatePrice()

{
  hash: '0xdbe3d17ab455e4184f4ede3b9f74a0192d2508a5a6bbd0d964538c4224eb205c',
  type: 0,
  accessList: null,
  blockHash: '0xdf729ef1100bf73bf660f5ae817e1842051c28fddd715aa7bae3b192eaaab4bd',
  blockNumber: 20001022,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x70a760ACD5503A8D6746BC7F00571F570ae0aD44',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 778,
  data: '0x673a7e28',
  r: '0x1dc64ef708a791304f06249c5b6a71927a31ad6b18648c9e51fde659f16c0eab',
  s: '0x042b94778e81812be26b0260018e13e48fdbce3be38e11b9f2e988923c29d745',
  v: 62709,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}

In [70]:
//Checking total supply of long F3 
synthF3Long.totalSupply()

BigNumber { _hex: '0x1b78e92b78118e7a5d6b', _isBigNumber: true }

In [71]:
//Checking total supply of alphaFLT
alphaFLT.totalSupply()

BigNumber { _hex: '0x0d1ec18243dc61f96d', _isBigNumber: true }

In [91]:
//Checking market value of long F3
ls.marketSideValueInPaymentToken(1,true)

1.1672288944970034e+23

In [92]:
(0x18b78f7dfa2132fb774a - 0x19a85a883fc741ab8afc) / 10**18

-4441.8491606092775

In [96]:
//Checking market value of short F3
ls.marketSideValueInPaymentToken(1,false)
0x12a2debba8d1573adcb2 / 10**18

88007.01883890868

In [19]:
 (0x12a2def41f9b98800525 - 0x12af0cfc6353d5789fca) / 10**18

-224.67790446863268

In [50]:
treasury.paymentToken()

'0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063'

In [24]:
longShortFake.marketUpdateIndex.returns(3690)

In [26]:
longShortFake.marketUpdateIndex(2)

BigNumber { _hex: '0x0e6a', _isBigNumber: true }